In [3]:
import pdfplumber
import pandas as pd
import numpy as np
import os
import textEx
from datetime import datetime

In [45]:
def readAllYouChengPDF(dir:str) -> pd.DataFrame:
    column_names=['货品号', '规格', '产地', '品名', '仓库', '价钱', 'Notes','date']
    all_pages_df=pd.DataFrame(columns=column_names)
    for filename in os.listdir(dir):
            if filename.endswith('.pdf'):
                file_path = os.path.join(dir, filename)
                with pdfplumber.open(file_path) as pdf:
                    first_page = pdf.pages[0]
                    text = first_page.extract_text()
                    index_print_date = text.find("生效日期:")
                    if index_print_date != -1:
                        print_date = text[index_print_date + len("生效日期:"):]
                        date = print_date.split()[0]
                    else:
                        print("未找到列印日期")
                    contents=""
                    for page in pdf.pages:
                        contents += page.extract_text()
                        lines = contents.split('\n')
                        filtered_lines = []

                        for line in lines:
                            if line.strip().isdigit() or (line.strip() and line[0].isdigit()):
                                filtered_lines.append(line)
                            else:
                                continue
                        df = pd.DataFrame(filtered_lines, columns=['Filtered Content'])
                        split_columns = df['Filtered Content'].str.split(expand=True)
                        df['Column1'] = split_columns[0]
                        df['Column2'] = split_columns[1]
                        df['Column3'] = split_columns[2]
                        df['Column5'] = split_columns.iloc[:,3:].apply(lambda x: ' '.join(x.dropna()), axis=1)
                        separator_list = ['光一', '亞洲', '威強', '沙二', '其士', '送貨', '威', '沙二', '威強嘉里', '威嘉2', '振興倉', '嘉威']
                        def split_column(row):
                            for separator in separator_list:
                                if separator in row['Column5']:
                                    parts = row['Column5'].split(separator)
                                    return pd.Series([parts[0], separator + ''.join(parts[1:])])

                        df[['Column6', 'Column7']] = df.apply(split_column, axis=1)
                        to_drop = ['1082','2033A','2031A','1057','1056','1028','1058','1065','1063','1064','3062','3004','3031','3085','2041A']
                        df = df[~df['Column1'].isin(to_drop)]
                        df = df[df['Column2'] != '運輸費']
                        split_columns_new = df['Column7'].str.split(expand=True)
                        df['仓库'] = split_columns_new[0]
                        df['价钱'] = split_columns_new[1]
                        df['Notes'] = split_columns_new[2]
                        df_selected = df.loc[:, ['Column1', 'Column2', 'Column3', 'Column6', '仓库', '价钱', 'Notes']]
                        df_selected['date'] = date
                        df_selected.columns = column_names
                        try:
                            all_pages_df = pd.concat([all_pages_df, df_selected], ignore_index=True)
                        except:
                            print(f"wrong {filename}")
    return all_pages_df

In [46]:
df=readAllYouChengPDF(r"quotation\有成")

In [47]:
df

,货品号,规格,产地,品名,仓库,价钱,Notes,date
0,1022,25KG,中國,特品 豬面肉 (150g+),光一,$33.5/KG,新,2023/6/29
1,1021,25KG,中國,精品 豬面肉 (250-300g),光一,$35/KG,新,2023/6/29
2,1012,10KG,中國,豬大腸頭,光一,$26/KG,None,2023/6/29
3,1070,20KG,中國,- 豬寸骨,光一,$54/KG,None,2023/6/29
4,1001D,25KG,中國,優質 梅肉,亞洲,$30/KG,None,2023/6/29
...,...,...,...,...,...,...,...,...
3360,5030,7.34KG,中國,兔仔牌 春卷 (6盒x72條),光一,$158/件,None,2023/7/25
3361,5032,7.65KG,中國,兔仔牌 咖喱角 (6盒x102隻),光一,$165/件,None,2023/7/25
3362,3235,6KG,中國,千鶴 日式雞肉餃子 (20Gx30隻x10包),光一,$260/件,None,2023/7/25
3363,5201,12KG,中國,兔仔牌 粟米粒 (12包x1K),光一,$5.7/LB,None,2023/7/25


In [54]:
def _specToList(spec :list) -> list:

    # Sort 'spec' in alphabetical order
    spec.sort()

    # Initialize variables with empty strings
    spec1 = spec2 = spec3 = spec4 = spec5 = spec6 = ""

    # Assign values from 'spec' to 'spec1' to 'spec6' based on the length of 'spec'
    if len(spec) > 0:
        spec1 = spec[0]
    if len(spec) > 1:
        spec2 = spec[1]
    if len(spec) > 2:
        spec3 = spec[2]
    if len(spec) > 3:
        spec4 = spec[3]
    if len(spec) > 4:
        spec5 = spec[4]
    if len(spec) > 5:
        spec6 = spec[5]

    return [spec1,spec2,spec3,spec4,spec5,spec6]
        

def getYouChengQuote(df: pd.DataFrame) -> pd.DataFrame:
    
    df['specs'] = df['品名'].astype(str).apply(textEx.getSpec)
    df['brand'] = df['品名'].astype(str).apply(textEx.getBrand)
    for index, row in df.iterrows():
        if row['brand'] is not None:
            df.at[index, 'productName'] = row['品名'].replace(row['brand'], '')
        else:
            df.at[index, 'productName'] = row['品名']
    df['productTag'] = df['品名'].astype(str).apply(textEx.getProduct)
    df['supplier'] = '有成'
    df['category'] = df['品名'].astype(str).apply(textEx.getCategory)
    df['packing']=df['规格']
    df['origin'] = df['产地'].astype(str).apply(textEx.getCountry)
    df['effectiveDate'] = df['date']
    df[['spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6']] = df['specs'].apply(lambda x: pd.Series(_specToList(x)))
    df['price'] = df['价钱'].str.replace('$','').replace(" ","")
    df['weightUnit'] = df['价钱'].astype(str).apply(textEx.getWeightUnit)
    df['warehouse']=df['仓库'].astype(str).apply(textEx.getWarehoue)
    df['notes']= df['Notes'].astype(str)

    df = df.explode('warehouse')
    df['warehouse'] = df['warehouse'].astype(str)

    required_columns = [
    'productName', 'productTag', 'supplier', 'category', 'packing',
    'origin', 'brand', 'effectiveDate', 'spec1', 'spec2', 'spec3',
    'spec4', 'spec5', 'spec6', 'price', 'weightUnit', 'warehouse', 'notes'
    ]
    df=df[required_columns]
    return df

In [55]:
dfout=getYouChengQuote(df)

c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataF

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataF

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataF

...

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataF

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataF

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is

In [57]:
dfout.to_csv("有成.csv")